In [1]:
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

client = Client(API_KEY, API_SECRET)

In [3]:
with open('pairs.txt','r') as file:
    for line in file:
        symbols = line.split(',')

In [4]:
info = client.get_ticker()
d = {}
for i in info:
    d[i['symbol']] = float(i['priceChangePercent'])
d2 = {}
for i in d:
    if i in symbols:
        d2[i] = d[i]
d2 = {x[0] : x[1] for x in sorted(d2.items(), key=lambda x: x[1], reverse=True)}
for x in d2.items():
    if x[1] > 10:
        print(x[0],x[1])


GMTUSDT 37.28
MTLUSDT 28.749
DREPUSDT 20.765
TCTUSDT 18.473
FXSUSDT 16.234
LINAUSDT 15.96
OCEANUSDT 15.905
RUNEUSDT 15.89
NBTUSDT 15.621
DARUSDT 14.873
RENUSDT 14.622
LUNAUSDT 14.192
ZECUSDT 14.1
SANDUSDT 13.861
BNBUPUSDT 13.699
ADAUPUSDT 13.62
ETHUPUSDT 13.534
LINKUPUSDT 13.5
JOEUSDT 13.472
MULTIUSDT 13.428
BELUSDT 12.725
SXPUSDT 12.551
PYRUSDT 12.4
NEARUSDT 12.339
IMXUSDT 12.232
SNXUSDT 12.094
ICXUSDT 12.006
LRCUSDT 11.555
BTSUSDT 11.359
PLAUSDT 10.965
JASMYUSDT 10.857
BTCUPUSDT 10.723
OGNUSDT 10.655
BSWUSDT 10.512
APEUSDT 10.509
ASTRUSDT 10.15
ACAUSDT 10.029
SYSUSDT 10.027


In [ ]:
# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")

# fetch weekly klines since it listed
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")

# [
#     1499040000000,      // 0 Open time
#     "0.01634790",       // 1 Open
#     "0.80000000",       // 2 High
#     "0.01575800",       // 3 Low
#     "0.01577100",       // 4 Close
#     "148976.11427815",  // 5 Volume
#     1499644799999,      // 6 Close time
#     "2434.19055334",    // 7 Quote asset volume
#     308,                // 8 Number of trades
#     "1756.87402397",    // 9 Taker buy base asset volume
#     "28.46694368",      // 10 Taker buy quote asset volume
#     "17928899.62484339" // 11 Ignore.
# ]

In [ ]:
klines=[[1650351360000, '3.25100000', '3.25213000', '3.23643000', '3.23725000', '136298.00000000', 1650351419999, '442179.08394800', 783, '37671.50000000', '122218.22250800', '0'],
[1650372900000, '3.29000000', '3.30000000', '3.29000000', '3.29532000', '182334.60000000', 1650372959999, '600759.29294500', 681, '162474.80000000', '535306.09749100', '0']]

In [86]:
# Method for getting price
def get_price(ls):
    avg = (float(ls[1])+float(ls[4]))/2
    price = float("{:.4f}".format(avg))
    return price

# Method for getting percent change in 6 hour
def get_percent_change(ls_1,ls_2):
    price_1 = get_price(ls_1)
    price_2 = get_price(ls_2)
    
    change = (price_2 - price_1)/price_1
    
    change_in_percent = float("{:.4f}".format(change))*100
    
    return change_in_percent

In [87]:
from datetime import datetime, timedelta
klines = client.get_historical_klines("GMTUSDT", Client.KLINE_INTERVAL_1MINUTE,str(datetime.now() - timedelta(hours=12)), str(datetime.now()))
print(get_percent_change(klines[0],klines[-1]))
# for lines in klines:
#     print(get_time(lines),get_price(lines))
#     print('\n')

[1650353280000, '3.28140000', '3.28664000', '3.27467000', '3.27524000', '89197.00000000', 1650353339999, '292686.58601000', 650, '36617.80000000', '120172.29671500', '0']
[1650374820000, '3.28953000', '3.29217000', '3.28773000', '3.28773000', '9012.70000000', 1650374879999, '29650.66075700', 73, '1647.20000000', '5419.46910900', '0']


0.31


In [75]:
from datetime import datetime, timedelta
print(str(datetime.now() - timedelta(days=1)))

2022-04-18 18:19:07.716449


In [61]:
for i in klines:
    print(i)

[1648512000000, '0.00912600', '0.00922100', '0.00912000', '0.00919100', '33485.02900000', 1648555199999, '306.84534432', 24123, '16956.93100000', '155.40880110', '0']
[1648555200000, '0.00919200', '0.00922700', '0.00911500', '0.00915500', '43299.32600000', 1648598399999, '396.60893009', 27090, '22240.24000000', '203.73826752', '0']
[1648598400000, '0.00915500', '0.00928900', '0.00910100', '0.00928400', '44239.16000000', 1648641599999, '406.35709700', 31007, '23472.24300000', '215.66384180', '0']


In [ ]:
# The following code was in code cell no 2
# Creating Socket Manager
bsm = BinanceSocketManager(client)

# Creating a socket for BTC-USDT pair
# socket = bsm.trade_socket('BTCUSDT')
socket = bsm.trade_socket(PAIR)